In [1]:
import pandas
import numpy
import json
import networkx
from scipy.stats import truncnorm
from uuid import uuid4

# Generate behaviours

In [2]:
behaviours_df = pandas.DataFrame({
    "name": ["Walk", "Cycle", "PT", "Drive"]
})

Now generate the UUIDs.

In [3]:
behaviours_df["uuid"] = [str(uuid4()) for i in range(len(behaviours_df))]

In [4]:
behaviours_df.to_json("behaviours.json", orient="records")

# Generate beliefs

In [5]:
beliefs_df = pandas.DataFrame({
    "name": [
        "I care about the environment",
        "I want to get to work quickly",
        "I care about the social importance of the car",
        "I want to keep fit",
        "I do not want to perform exercise on my commute"
    ]
})

In [6]:
belief_uuids = [str(uuid4()) for i in range(len(beliefs_df))]
beliefs_df["uuid"] = belief_uuids

In [7]:
w_uuid = behaviours_df[behaviours_df["name"] == "Walk"].reset_index().uuid[0]
c_uuid = behaviours_df[behaviours_df["name"] == "Cycle"].reset_index().uuid[0]
p_uuid = behaviours_df[behaviours_df["name"] == "PT"].reset_index().uuid[0]
d_uuid = behaviours_df[behaviours_df["name"] == "Drive"].reset_index().uuid[0]

In [8]:
perceptions = [
    {
        w_uuid: 0.6,
        c_uuid: 0.7,
        p_uuid: 0.4,
        d_uuid: -0.9
    }, {
        w_uuid: -0.5,
        c_uuid: 0.0,
        p_uuid: -0.1,
        d_uuid: 0.4
    }, {
        w_uuid: 0.0,
        c_uuid: 0.0,
        p_uuid: -0.3,
        d_uuid: 0.8
    }, {
        w_uuid: 0.2,
        c_uuid: 0.7,
        p_uuid: 0.0,
        d_uuid: 0.0
    }, {
        w_uuid: -0.2,
        c_uuid: -0.7,
        p_uuid: 0.1,
        d_uuid: 0.2
    }
]

In [9]:
beliefs_df["perceptions"] = perceptions

In [10]:
relationships = [
    {
        belief_uuids[2]: -0.5,
        belief_uuids[3]: 0.2
    }, {
        belief_uuids[0]: -0.1,
        belief_uuids[2]: 0.3
    }, {
        belief_uuids[0]: -0.3,
        belief_uuids[1]: 0.7,
        belief_uuids[4]: 0.5
    }, {
        belief_uuids[0]: 0.2,
        belief_uuids[1]: -0.1,
        belief_uuids[4]: -1.0
    }, {
        belief_uuids[0]: -0.1,
        belief_uuids[1]: 0.4,
        belief_uuids[2]: 0.1,
        belief_uuids[3]: -1.0
    }
]

In [11]:
beliefs_df["relationships"] = relationships

In [12]:
beliefs_df.to_json("beliefs.json", orient="records")

# Generate PRS

In [13]:
# beliefs x behaviours matrix
# behaviours: walk, cycle, PT, drive
# beliefs: see above
prs_mat = numpy.matrix([
    [ 0.7,  0.8,  0.4,  0.5],
    [-0.5, -0.5,  0.9, -0.2],
    [ 0.0,  0.0,  1.0, -0.5],
    [ 0.5,  0.9, -0.2,  0.0],
    [-0.2, -0.8,  0.4,  0.4]
])

In [14]:
prs = [{
    "beliefUuid": beliefs_df.uuid.values[i],
    "behaviourUuid": behaviours_df.uuid.values[j],
    "value": prs_mat[i, j]
} for i in range(prs_mat.shape[0]) for j in range(prs_mat.shape[1])]

In [15]:
with open("prs.json", "w") as outfile:
    json.dump(prs, outfile)

# Generate agents
## Start with UUIDs

In [16]:
agents_df = pandas.DataFrame(
    {"uuid": [str(uuid4()) for i in range(500)]}
)

## Generate friends
We'll use a small-world network

In [17]:
network = networkx.watts_strogatz_graph(
    n = len(agents_df),
    k = 10,
    p = 0.3
)

Now, let's say there's an 80% chance of being friends with yourself

In [18]:
rng = numpy.random.default_rng()
for i in range(len(agents_df)):
    if rng.random() <= 0.8:
        network.add_edge(i, i)

Now let's add some weights

In [19]:
a, b = (0 - 0) / 1, (1 - 0) / 1
for edge in network.edges:
    network.edges[edge[0], edge[1]]["weight"] = truncnorm.rvs(a, b)

Now let's add it do the dataframe

In [20]:
friends = numpy.full(len(agents_df), {})
for edge in network.edges:
    friends[edge[0]][agents_df.iloc[edge[1]]["uuid"]] =\
         network.edges[edge[0], edge[1]]["weight"]

In [21]:
agents_df["friends"] = friends

Now we can generate the deltas

In [22]:
deltas_mat = rng.normal(loc=1.0-0.001, scale=0.1,size=(len(agents_df), len(beliefs_df)))
# Ensure fuly positive (very unlikely for this not to be the case)
deltas_mat = numpy.abs(deltas_mat) + 0.001

In [23]:
agents_df["deltas"] = [
    {beliefs_df.iloc[belief_i]["uuid"]: deltas_mat[agent_i, belief_i]
        for belief_i in range(deltas_mat.shape[1])}
    for agent_i in range(deltas_mat.shape[0])
]

## Generate activations

Now we can generate the activations, we'll use a two part model where there is a 50% chance of activation being non-zero and then a truncated Normal(0, 0.1) distribution capped at -1 and +1

In [24]:
def random_activation():
    a, b = (-1 - 0) / 1, (1 - 0) / 1
    rng = numpy.random.default_rng()
    if rng.random() <= 0.5:
        return 0.0
    else:
        return truncnorm.rvs(a, b)

agents_df["activations"] = [
    {0: {belief_uuid: random_activation()
        for belief_uuid in beliefs_df.uuid.values}}
    for _i in range(len(agents_df))
]

Finally, we can generate the initial actions

In [25]:
def choose_initial_actions(
    activations,
    prs
):
    return numpy.array(numpy.argmax(activations * prs, axis=1)).reshape(activations.shape[0])

In [26]:
initial_actions = choose_initial_actions(
    numpy.array([
        [agents_df.iloc[agent_i]["activations"][0][belief_uuid]
            for belief_uuid in beliefs_df.uuid.values]
    for agent_i in range(len(agents_df))]),
    prs_mat
)

agents_df["actions"] = [
    {
        0: behaviours_df.iloc[initial_actions[i]].uuid
    } for i in range(len(agents_df))
]

In [27]:
agents_df.to_json("agents.json", orient="records")